In [1]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pprint
import sys

In [2]:
base_url="https://www.foxsports.com/soccer/stats?competition=2&season=20180&category=STANDARD&pos=0&team=0&isOpp=0&sort=3&sortOrder=0&page="
player_list=[]
player_goals=[]
player_SOG=[]
player_shoot=[]
player_club=[]
club_acronym=[]

In [3]:
for i in range (1,10):
    url=base_url+str(i)
    response=requests.get(url)
    soup=bs(response.text, "html.parser")
    player_name_list_box=soup.find_all("a", class_="wisbb_fullPlayer")
    player_goals_box=soup.find_all("td", class_="wisbb_priorityColumn wisbb_selected")
    player_sog_box=soup.find_all("td", class_="wisbb_priorityColumn")
    player_shoot_list=soup.find_all("td", class_=None)
    player_club_box=soup.find_all("span",class_="wisbb_tableAbbrevLink")
    
    #player name
    for player in player_name_list_box:
        player_list.append(player.find("span").text)
        
    #player goals
    for goal in player_goals_box:
        player_goals.append(goal.text)
        
    #player shoot on goals
    for i in range(1,len(player_sog_box)+1):
        if not i%4:
            player_SOG.append(player_sog_box[i-1].text)
            
    #player shoots
    for i in range(1, len(player_shoot_list)+1):
        if i%5==3:
            player_shoot.append(player_shoot_list[i-1].text)
            
    #player club acronym, if player has no club (50 examples), use "None"
    for club in player_club_box:
        if len(list(club.text.replace("\n","")))<3:
            player_club.append("None")
        else:
            player_club.append(club.text.replace("\n","").strip())
    

In [4]:
df=pd.DataFrame({"FullName":player_list,
                 "Goals":player_goals,
                 "ShootOnGoals":player_SOG,
                 "Shoots":player_shoot,
                 "PlayerClub":player_club
                            })

In [5]:
last_name=[]
first_name=[]

In [6]:
#extract first name and last name, if only first name is shown (for some cases), use "Unknown" for last name
for name in df["FullName"]:
    if len(name.split(","))==2:
        last_name.append(name.split(",")[0].strip()) 
        first_name.append(name.split(",")[1].strip())
    if len(name.split(","))==1:
        last_name.append("Unknown")
        first_name.append(name.split(",")[0].strip())

In [7]:
df["LastName"]=last_name
df["FirstName"]=first_name

In [8]:
club_acronym=list(set(player_club))
club_acronym.remove("None")
print(club_acronym)

['VLD', 'ESP', 'ATL', 'RV', 'LGN', 'VIL', 'GET', 'HUE', 'EIB', 'VAL', 'SEV', 'BET', 'CEL', 'RSO', 'RMD', 'BAR', 'ALA', 'ATH', 'GIR', 'LEV']


In [9]:
dic_club1={
        "LGN":"CD Leganés",
        "ATH":"Athletic Club",
        "ALA":"D. Alavés",
        "CEL":"RC Celta",
        "BET":"Real Betis",
        "VIL":"Villarreal CF",
        "GIR":"Girona FC",
        "LEV":"Levante UD",
        "ESP":"RCD Espanyol",
        "GET":"Getafe CF",
        "EIB":"SD Eibar",
        "RMD":"Real Madrid",
        "BAR":"FC Barcelona",
        "VAL":"Valencia CF",
        "HUE":"SD Huesca",
        "SEV":"Sevilla FC",
        "VLD":"R. Valladolid CF",
        "RV":"Rayo Vallecano",
        "ATL":"Atlético Madrid",
        "RSO":"Real Sociedad",
}

In [10]:
df['ForKey'] = df['FirstName'].str[:3] + df['LastName'].str[:3] + df['PlayerClub'].str[:3]
df.to_csv("Performance_La_Liga.csv")

In [11]:
#continue with API
df.head()

,FullName,Goals,ShootOnGoals,Shoots,PlayerClub,LastName,FirstName,ForKey
0,"Messi, Lionel",9,22,47,BAR,Messi,Lionel,LioMesBAR
1,"Suarez, Luis",9,21,45,BAR,Suarez,Luis,LuiSuaBAR
2,"Stuani, Cristhian",8,11,20,GIR,Stuani,Cristhian,CriStuGIR
3,"Aspas, Iago",8,15,29,CEL,Aspas,Iago,IagAspCEL
4,"Iglesias, Borja",7,18,32,ESP,Iglesias,Borja,BorIglESP


In [12]:
url = "https://www.easports.com/fifa/ultimate-team/api/fut/item?"
league = "53"
page = 1
query_url = f"{url}league={league}&page={page}"

# Get FIFA Ultimate-team data
fifa_response = requests.get(query_url)
fifa_json = fifa_response.json()
totalPages = fifa_json["totalPages"]
playersCount = fifa_json["count"]
#playersCount

In [13]:
# Structuring DataFrame0
#[players.append(fifa_json["items"][i]["firstName"]) for i in range(playersCount)]
players2 = []
for p in range(totalPages):
    pg = p+1
    query_url = f"{url}league={league}&page={pg}"
    fifa_response = requests.get(query_url)
    fifa_json = fifa_response.json()
    playersCount = fifa_json["count"]
    for i in range(playersCount):
        players2.append([fifa_json["items"][i]["firstName"],fifa_json["items"][i]["lastName"],fifa_json["items"][i]["nation"]["name"],fifa_json["items"][i]["club"]["name"],fifa_json["items"][i]["positionFull"],fifa_json["items"][i]["age"],fifa_json["items"][i]["acceleration"],fifa_json["items"][i]["aggression"],fifa_json["items"][i]["finishing"],fifa_json["items"][i]["freekickaccuracy"],fifa_json["items"][i]["longpassing"],fifa_json["items"][i]["longshots"],fifa_json["items"][i]["penalties"],fifa_json["items"][i]["shortpassing"],fifa_json["items"][i]["sprintspeed"],fifa_json["items"][i]["quality"]])
players = pd.DataFrame(players2, columns=['FirstName','LastName','Country','Club','Position','Age','Acceleration','Aggression','Finishing','FreeKickAccuracy','LongPassing','LongShots','Penalties','ShortPassing','SprintSpeed','Quality'])


In [14]:
#make a list of full club name
club_fullname=[]
for k,v in dic_club1.items():
    club_fullname.append(v)

In [15]:
#only select clubs in La Liga
df_player=players.loc[players["Club"].isin(club_fullname)]

In [16]:
#replace the club full name with acronym
dic_club2={
        "CD Leganés":"LGN",
        "Athletic Club":"ATH",
        "D. Alavés":"ALA",
        "RC Celta":"CEL",
        "Real Betis":"BET",
        "Villarreal CF":"VIL",
        "Girona FC":"GIR",
        "Levante UD":"LEV",
        "RCD Espanyol":"ESP",
        "Getafe CF":"GET",
        "SD Eibar":"EIB",
        "Real Madrid":"RMD",
        "FC Barcelona":"BAR",
        "Valencia CF":"VAL",
        "SD Huesca":"HUE",
        "Sevilla FC":"SEV",
        "R. Valladolid CF":"VLD",
        "Rayo Vallecano":"RV",
        "Atlético Madrid":"ATL",
        "Real Sociedad":"RSO",
}

df_player["Club"]=df_player["Club"].apply(lambda x:dic_club2[x])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
df_player['ForKey'] = df_player['FirstName'].str[:3] + df_player['LastName'].str[:3] + df_player['Club'].str[:3]
df_player.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,FirstName,LastName,Country,Club,Position,Age,Acceleration,Aggression,Finishing,FreeKickAccuracy,LongPassing,LongShots,Penalties,ShortPassing,SprintSpeed,Quality,ForKey
0,Luka,Modrić,Croatia,RMD,Centre Midfielder,33,94,83,89,85,96,99,99,99,85,gold,LukModRMD
1,Lionel,Messi,Argentina,BAR,Centre Forward,31,99,66,99,99,96,99,81,99,94,gold,LioMesBAR
2,Sergio,Ramos García,Spain,RMD,Centre Back,32,86,96,76,83,90,69,86,97,85,gold,SerRamRMD
3,Raphaël,Varane,France,RMD,Centre Back,25,88,88,62,72,99,82,66,99,91,gold,RapVarRMD
4,Lionel,Messi,Argentina,BAR,Striker,31,93,51,97,98,90,96,76,94,87,gold,LioMesBAR


In [18]:
df_merged=pd.merge(df, df_player, on='ForKey', how='outer')
df_nonull=df_merged.dropna(how="any")

In [19]:
df_clean=df_nonull.to_csv("Cleaned FIFA and Foxsports.csv")